# 가상환경 torch_cuda_yolo_env

In [1]:
import torch
print("torch:", torch.__version__, "cuda:", torch.version.cuda, "is_available:", torch.cuda.is_available())

torch: 2.6.0+cu124 cuda: 12.4 is_available: True


In [3]:
import os
from dotenv import load_dotenv
loaded = load_dotenv()

In [4]:
from inference_sdk import InferenceHTTPClient
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=os.getenv("ROBOFLOW_API_KEY")
)
CLIENT

In [5]:
result = CLIENT.infer('sample.png',model_id='rock-paper-scissors-sxsw/14')
result

{'inference_id': 'f9eb2012-6c6d-4947-ae8f-a87b7efc363b',
 'time': 0.0398653309994188,
 'image': {'width': 620, 'height': 413},
 'predictions': [{'x': 145.0,
   'y': 238.0,
   'width': 148.0,
   'height': 132.0,
   'confidence': 0.8332926630973816,
   'class': 'Rock',
   'class_id': 1,
   'detection_id': 'dd06976b-7f80-4d15-b4be-acb900317a83'},
  {'x': 444.0,
   'y': 166.5,
   'width': 318.0,
   'height': 129.0,
   'confidence': 0.7048391103744507,
   'class': 'Scissors',
   'class_id': 2,
   'detection_id': '167bcb59-8567-4e01-b888-94b35cde2a2c'}]}

In [ ]:
for pred in result['predictions']:
    x = pred['x']
    y = pred['y']
    width = pred['width']
    height = pred['height']
    conf = pred['confidence']
    obj_class = pred['class']
    print(x,y,width,height,conf,obj_class)